In [1]:
import numpy as np
import pandas as pd

In [2]:
with open("hashcode.in") as f:
    D, I, S, V, F = tuple(int(temp) for temp in f.readline().split())
    print(D)
    street_info = {}
    for temp in range(S):
        street_info[temp]=tuple(temp for temp in f.readline().split())
    car_info = {}
    for temp in range(V):
        car_info[temp]=tuple(temp for temp in f.readline().split())

street_info = pd.DataFrame.from_dict(street_info, orient = 'index',columns = ['start_intersection','end_intersection','street_name','travel_time']).set_index('street_name')
car_info = pd.DataFrame.from_dict(car_info, orient = 'index',columns = ['n_streets'] + ['street_'+ str(i) for i in range(120)])

7071


In [3]:
start_intersection = street_info.loc[:,'start_intersection']
end_intersection = street_info.loc[:,'end_intersection']

In [4]:
car_info.iloc[:,1:].apply(lambda col: col.apply(lambda v: start_intersection[v]),axis= 0)

,street_0,street_1,street_2,street_3,street_4,street_5,street_6,street_7,street_8,street_9,...,street_110,street_111,street_112,street_113,street_114,street_115,street_116,street_117,street_118,street_119
0,5853,1412,65,1007,3970,6022,8,61,3954,748,...,1597,4709,3738,623,95,11,5803,1258,57,1656
1,2978,32,550,261,2222,7679,6073,4763,5666,426,...,4517,64,2712,1314,609,1274,6,3226,5365,10
2,6994,479,58,1260,28,3673,4299,4488,1158,1417,...,162,4966,240,2194,8,1288,1252,896,621,4570
3,7663,4451,1671,6514,7768,4,5322,1576,5808,5890,...,24,4806,440,621,896,3609,103,4307,4091,124
4,3947,7026,1166,262,3548,1181,6103,882,218,1420,...,1461,4820,5411,3607,5984,1469,63,7525,151,4636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3535,2231,4947,39,3384,5430,7361,1322,3659,246,...,2273,5597,6219,4386,98,2184,1311,758,71,3239
996,760,1162,7247,8,471,7149,1,210,535,7,...,5656,2110,518,2031,2821,685,3074,1475,190,5756
997,3793,3456,564,4945,127,2629,38,1205,1154,1408,...,1256,5989,120,1565,1487,650,6413,3421,3504,4927
998,7259,12,2371,213,7651,7169,62,4549,1059,1252,...,41,5,4248,3434,2617,6890,4908,5678,1858,74


In [5]:
street_info['end_light_schedule'] = np.ones(street_info.shape[0],).astype('int')

In [6]:
traffic_light_info = pd.DataFrame(street_info.groupby('end_intersection').apply(lambda x: list(x.index)), columns = ['entering_from'])
traffic_light_info['exit_to'] = street_info.groupby('start_intersection').apply(lambda x: list(x.index))
traffic_light_info['schedule'] = traffic_light_info['entering_from'].apply(lambda lst: [1 for i in lst])
traffic_light_info['schedule_pairs'] = traffic_light_info.apply(lambda row: dict(zip(row['entering_from'],row['schedule'])), axis = 1)

In [20]:
class traffic_light:
    def __init__(self, traffic_light_id = None, entering_roads = [], exiting_roads = [], schedule_pairs = {}):
        self.traffic_light_id = traffic_light_id
        self.entering_roads = entering_roads
        self.exiting_roads = exiting_roads
        self.schedule_pairs = schedule_pairs #a dict
        self.schedule_queue = []
    def initialize_as_one(self):
        for road in self.entering_roads:
            self.schedule_pairs[road] = 1
            self.schedule_queue.append(road)
    def initialize(self):
        for road, elapse in self.schedule_pairs.items():
            self.schedule_queue.extend([road]*elapse)
    def sprint(self):
        print(self.schedule_queue)
    def passTime(self):
        temp = self.schedule_queue[0]
        self.schedule_queue = self.schedule_queue[1:] + [temp]
    def isGreenLight(self, enter_from = None):
        if enter_from is None:
            isGreenLight = None
        else:
            isGreenLight = enter_from == self.schedule_queue[0]
        #self.passTime()
        return isGreenLight

traffic_light_info['object'] = traffic_light_info.apply(lambda row: traffic_light(row.name, row.entering_from, row.exit_to, row.schedule_pairs), axis = 1)
traffic_light_info['object'].apply(lambda x: x.initialize())

end_intersection
0       None
1       None
10      None
100     None
1000    None
        ... 
995     None
996     None
997     None
998     None
999     None
Name: object, Length: 8000, dtype: object

In [70]:
class car:
    def __init__(self, car_id = None, path = [], path_pairs=[]):
        self.car_id = car_id
        self.path = path
        self.currentPosition = None
        #current location of the car (None for in the middle of road, road name for waiting at the end, 'finished' for finished)
        self.isWaiting = False
        self.path_pairs = path_pairs
        self.travel_queue = []
    def initialize(self):
        for road, elapse in self.path_pairs:
            self.travel_queue.extend([None] * (int(elapse)-1) + [road])
    def sprint(self):
        print(self.travel_queue)
    def passing(self, isTraveling = True): #WIP car time pass, 
        print('isTraveling', isTraveling)
        if isTraveling:
            if self.travel_queue == []:
                self.currentPosition = 'Finished'
                return 'Finished'
            else:
                self.currentPosition = self.travel_queue[0]
                self.travel_queue = self.travel_queue[1:]
                print('passed')
                return None
        else:
            return None
car_info['path'] = car_info.apply(lambda x: list(x[1:121]), axis = 1)
car_info['path_pairs'] = car_info['path'].apply(lambda lst: [tuple([i,street_info['travel_time'][i]]) for i in lst])
car_info['object'] = car_info.apply(lambda row: car(row.name, row.path, row.path_pairs), axis = 1)
car_info['object'].apply(lambda x: x.initialize())

0      None
1      None
2      None
3      None
4      None
       ... 
995    None
996    None
997    None
998    None
999    None
Name: object, Length: 1000, dtype: object

# WIP part

In [71]:
c = car(car_info.index[0], car_info['path'][0][:1], car_info['path_pairs'][0][:1])
c.initialize()

print(c.travel_queue[:5])
def checkPassage(car_object, traffic_light_info, street_info):
    print('checking car {} passing  {}'.format(car_object.car_id, car_object.currentPosition))
    if car_object.currentPosition is None:
        canTravel = True
    else:
        canTravel = traffic_light_info['object'][street_info['end_intersection'][car_object.currentPosition]].isGreenLight(car_object.currentPosition)
    #print('can travel', canTravel)
    print(car_object.passing(isTraveling = canTravel))
    return canTravel
def timePlusOne(traffic_light_info):
    traffic_light_info['object'].apply(lambda l: l.passTime())
while c.currentPosition  != 'Finished':
    checkPassage(c, traffic_light_info, street_info)
    timePlusOne(traffic_light_info)
print(c.travel_queue[:5])

[None, 'fifd-bebc']
checking car 0 passing  None
isTraveling True
passed
None
checking car 0 passing  None
isTraveling True
passed
None
checking car 0 passing  fifd-bebc
isTraveling False
None
checking car 0 passing  fifd-bebc
isTraveling False
None
checking car 0 passing  fifd-bebc
isTraveling False
None
checking car 0 passing  fifd-bebc
isTraveling False
None
checking car 0 passing  fifd-bebc
isTraveling False
None
checking car 0 passing  fifd-bebc
isTraveling False
None
checking car 0 passing  fifd-bebc
isTraveling False
None
checking car 0 passing  fifd-bebc
isTraveling False
None
checking car 0 passing  fifd-bebc
isTraveling False
None
checking car 0 passing  fifd-bebc
isTraveling False
None
checking car 0 passing  fifd-bebc
isTraveling True
Finished
[]


In [67]:
c = car(car_info.index[0], car_info['path'][0][:1], car_info['path_pairs'][0][:1])
c.initialize()
c.travel_queue

[None, 'fifd-bebc']

In [60]:
for t in range(1000):
    print(traffic_light_info['object'][street_info['end_intersection'][c.currentPosition]].isGreenLight(c.currentPosition))
    timePlusOne(traffic_light_info)

False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [59]:
traffic_light_info['object'][street_info['end_intersection'][c.currentPosition]].isGreenLight(c.currentPosition)

False

In [24]:
if c.currentPosition is not None:
    print(street_info['end_intersection'][c.currentPosition]

SyntaxError: unexpected EOF while parsing (<ipython-input-24-0fd848e0f569>, line 2)

False